In [1]:
# %%
import sys
sys.path.append("..")
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Audio
import scipy.signal as sg
import librosa
from src.utilities.utils import babble_noise
from mcsm_benchs.Benchmark import Benchmark
from pesq import pesq
from pystoi import stoi
import os
from src.utilities.scale_functions import scale_fun_APF, scale_fun_Fvs, scale_fun_F

# from src.aps_metric.perf_metrics import musical_noise_measure_aps
from src.methods.method_hard_threshold import NewMethod as hard_thresholding
from src.methods.method_garrote_threshold import NewMethod as garrote_thresholding
from src.methods.method_delaunay_triangulation import delaunay_triangulation_denoising 


In [2]:

def dt_method(signal,*args,**kwargs):
    output = delaunay_triangulation_denoising(signal=signal,
                                            mcsim_path='src/utilities/simulations',
                                            *args,**kwargs)
    return output


In [3]:
# Create a dictionary of signals.
signals_dict = {}
N = 1*8192 # A bit more than a second of signal.

# for i in range(6,7):
#     x,fs = librosa.load('./signals/{}_male.mp3'.format(i), sr=8000)
#     x = x[:N]
#     signals_dict['{}_male'.format(i)]=x

for i in range(6,7):
    x,fs = librosa.load('./signals/{}_female.mp3'.format(i), sr=8000)
    x = x[:N]
    signals_dict['{}_female'.format(i)]=x

# x,fs = librosa.load('./signals/cello.wav', sr=8000)
# x = x[500:500+N]
# signals_dict['cello']=x

# x, fs = librosa.load(librosa.ex('trumpet'), duration=1.5, sr=8000)
# x = x[0:N]
# signals_dict['trumpet']=x



In [4]:
# Parameters
Nfft = 2*N
thr = np.arange(0.5,5.5,0.5)
lmax = np.arange(1.0,3.25,0.25)
SNRs = [-5, 0, 5, 10, 15, 20]
reps = 50

# %%
dict_methods = {
                'dt': dt_method,
                'ht': hard_thresholding().method,
                # 'st': garrote_thresholding().method,
                }

dict_parameters = {
                    'dt': [{'scale_fun':scale_fun_APF},
                           {'scale_fun':scale_fun_Fvs},
                           {'scale_fun':scale_fun_F},
                     #       *[{'LB':q} for q in lmax],
                           ],
                    'ht': [{'coeff':1.5, 'Nfft':2**12},],
                    # 'st': [{'coeff':q, 'Nfft':2**12} for q in thr],
                    }

In [5]:
# Once new methods are added, load previous benchmark and add new methods.
new_methods_flag = False
# benchmark = Benchmark.load_benchmark('./results/benchmark_babble')
# benchmark.add_new_method(methods=dict_methods,parameters=dict_parameters)


In [6]:
# Here define performance functions.
# PESQ:
def pesq_metric(x,x_hat,**kwargs):
    # print("It's using pesq!")
    return pesq(fs,x,x_hat,'nb')

# STOI:
def stoi_metric(x,x_hat,**kwargs):
    stoival = stoi(x, x_hat, fs, extended=False)
    return 100/(1+np.exp(-17.4906*stoival+9.6921))

from src.aps_metric.perf_metrics import musical_noise_measure_aps_8
def aps(x,xhat,**kwargs):
    return musical_noise_measure_aps_8(x,xhat,fs=fs,**kwargs)

print(stoi_metric(x,x), pesq_metric(x,x))

perf_funs = {'pesq':pesq_metric,
            #  'stoi':stoi_metric,
	         'aps':aps, # Only compute this at best pesq.
             }

if new_methods_flag:
    benchmark.objectiveFunction = perf_funs
    benchmark.verbosity = 5
    # benchmark.parallel_flag = 2

99.95898182754574 4.548638343811035


In [7]:
# %%
# np.random.seed(42)
if not new_methods_flag:
    benchmark = Benchmark(task = 'denoising',
                            methods = dict_methods,
                            N = len(x), 
                            SNRin = [5,], 
                            repetitions = reps,
                            signal_ids= signals_dict,
                            parameters=dict_parameters,
                            verbosity=5, 
                            parallelize=5,
                            obj_fun=perf_funs,
                            complex_noise=babble_noise,                 
                            )
# %%

Number of processors:  12
Parallel pool: 5


In [8]:
benchmark.run() # Run the benchmark

Running benchmark...
- Signal 6_female
-- SNR: 5 dB
--- Parallel loop -- Method: dt(all parameters)
--- Parallel loop -- Method: ht(all parameters)
------ Inner loop. dt: 0
------ Inner loop. dt: 10
------ Inner loop. dt: 20
------ Inner loop. dt: 30
------ Inner loop. dt: 40
Threshold:1.5428024847094646
Threshold:1.542559373474868
Threshold:1.5466982018598516
------ Inner loop. dt: 11
------ Inner loop. dt: 31
------ Inner loop. dt: 21
Threshold:1.564237946446299
Threshold:1.5370592945665218
------ Inner loop. dt: 1
------ Inner loop. dt: 41
Threshold:1.5425752044018628
Threshold:1.5681084343917704
------ Inner loop. dt: 12
------ Inner loop. dt: 22
Threshold:1.5431603899474882
Threshold:1.550458986737813
------ Inner loop. dt: 32
Threshold:1.5642614792711378
------ Inner loop. dt: 42
------ Inner loop. dt: 2
Threshold:1.5717968179513635
Threshold:1.5463209760159904
------ Inner loop. dt: 23
------ Inner loop. dt: 13
Threshold:1.56341331960702
------ Inner loop. dt: 33
Threshold:1.537

{'pesq': {'6_female': {5: {'dt': {"{'scale_fun': <function scale_fun_APF at 0x7a2e3076be20>}": [1.588511347770691,
      1.52431321144104,
      1.6752474308013916,
      1.6331546306610107,
      1.5095562934875488,
      1.5580956935882568,
      1.557686686515808,
      1.7065305709838867,
      1.7321499586105347,
      1.5953912734985352,
      1.5612099170684814,
      1.495649814605713,
      1.5786032676696777,
      1.5369750261306763,
      1.6761788129806519,
      1.4734102487564087,
      1.5039334297180176,
      1.4900730848312378,
      1.4637150764465332,
      1.5661157369613647,
      1.606764316558838,
      1.5548779964447021,
      1.5475889444351196,
      1.602329969406128,
      1.5957322120666504,
      1.5504510402679443,
      1.5636441707611084,
      1.502523422241211,
      1.6330008506774902,
      1.7460131645202637,
      1.553452491760254,
      1.6550097465515137,
      2.0460712909698486,
      1.5138137340545654,
      1.588418960571289,
      1.54

In [9]:
benchmark.save_to_file(filename = './results/benchmark_aps_3')

True

In [10]:
# This formats the results on a DataFrame
results_parameters = benchmark.get_results_as_df()
results_parameters

[    Method                                          Parameter Signal_id  \
 100     dt  {'scale_fun': <function scale_fun_APF at 0x7a2...  6_female   
 101     dt  {'scale_fun': <function scale_fun_APF at 0x7a2...  6_female   
 102     dt  {'scale_fun': <function scale_fun_APF at 0x7a2...  6_female   
 103     dt  {'scale_fun': <function scale_fun_APF at 0x7a2...  6_female   
 104     dt  {'scale_fun': <function scale_fun_APF at 0x7a2...  6_female   
 ..     ...                                                ...       ...   
 795     ht                       {'coeff': 1.5, 'Nfft': 4096}  6_female   
 796     ht                       {'coeff': 1.5, 'Nfft': 4096}  6_female   
 797     ht                       {'coeff': 1.5, 'Nfft': 4096}  6_female   
 798     ht                       {'coeff': 1.5, 'Nfft': 4096}  6_female   
 799     ht                       {'coeff': 1.5, 'Nfft': 4096}  6_female   
 
      Repetition          5  
 100           0  45.909156  
 101           1  32.09073